### Паттерн Стратегия

####  Пример 6.1. 
Реализация класса Order с помощью взаимозаменяемых стратегий предоставления скидки

In [6]:
from abc import ABC, abstractmethod
from collections import namedtuple

Customer = namedtuple('Customer', 'name fidelity')


class LineItem:
    def __init__(self, product, quantity, price):
        self.product = product
        self.quantity = quantity
        self.price = price

    def total(self):
        return self.price * self.quantity


class Order:  # Context
    def __init__(self, customer, cart, promotion=None):
        self.customer = customer
        self.cart = list(cart)
        self.promotion = promotion

    def total(self):
        if not hasattr(self, '__total'):
            self.__total = sum(item.total() for item in self.cart)
        return self.__total

    def due(self):
        if self.promotion is None:
            discount = 0
        else:
            discount = self.promotion.discount(self)
        return self.total() - discount

    def __repr__(self):
        fmt = '<Order total: {:.2f} due: {:.2f}>'
        return fmt.format(self.total(), self.due())


class Promotion(ABC):  # Abstract base class
    @abstractmethod
    def discount(self, order):
        """Return discount as a positive dollar amount"""


class FidelityPromo(Promotion):  # First concrete strategy
    """5 procents for customers with at least 1000 loyalty points"""

    def discount(self, order):
        return order.total() * .05 if order.customer.fidelity >= 1000 else 0


class BulkItemPromo(Promotion):  # Second concrete strategy
    """10% discount for each LineItem with a minimum order of 20 units"""

    def discount(self, order):
        discount = 0
        for item in order.cart:
            if item.quantity >= 20:
                discount += item.total() * .1
        return discount


class LargeOrderPromo(Promotion):  # Third concrete strategy
    """7% discount for orders including at least 10 different items"""

    def discount(self, order):
        distinct_items = {item.product for item in order.cart}
        if len(distinct_items) >= 10:
            return order.total() * .07
        return 0

#### Пример 6.2. 
Пример использования класса Order с различными стратегиями скидок

In [10]:
joe = Customer('John Doe', 0)
ann = Customer('Ann Smith', 1100)
cart = [LineItem('banana', 4, .5),
        LineItem('apple', 10, 1.5),
        LineItem('watermelon', 5, 5.0)]
Order(joe, cart, FidelityPromo())

<Order total: 42.00 due: 42.00>

In [8]:
Order(ann, cart, FidelityPromo())

<Order total: 42.00 due: 39.90>

In [9]:
banana_cart = [LineItem('banana', 30, .5),
               LineItem('apple', 10, 1.5)]
Order(joe, banana_cart, BulkItemPromo())

<Order total: 30.00 due: 28.50>

In [11]:
long_order = [LineItem(str(item_code), 1, 1.0) for item_code in range(10)]
Order(joe, long_order, LargeOrderPromo())

<Order total: 10.00 due: 9.30>

In [12]:
Order(joe, cart, LargeOrderPromo())

<Order total: 42.00 due: 42.00>

#### Пример 6.3. 
Класс Order, в котором стратегии предоставления скидок реализованы
в виде функций

In [5]:
from collections import namedtuple

Customer = namedtuple('Customer', 'name fidelity')


class LineItem:
    def __init__(self, product, quantity, price):
        self.product = product
        self.quantity = quantity
        self.price = price

    def total(self):
        return self.price * self.quantity


class Order:  # Context
    def __init__(self, customer, cart, promotion=None):
        self.customer = customer
        self.cart = list(cart)
        self.promotion = promotion

    def total(self):
        if not hasattr(self, '__total'):
            self.__total = sum(item.total() for item in self.cart)
        return self.__total

    def due(self):
        if self.promotion is None:
            discount = 0
        else:
            discount = self.promotion(self)
        return self.total() - discount

    def __repr__(self):
        fmt = '<Order total: {:.2f} due: {:.2f}>'
        return fmt.format(self.total(), self.due())


def fidelity_promo(order):
        """5 procents for customers with at least 1000 loyalty points"""
        return order.total() * .05 if order.customer.fidelity >= 1000 else 0


def bulk_item_promo(order):
        """10% discount for each LineItem with a minimum order of 20 units"""
        discount = 0
        for item in order.cart:
            if item.quantity >= 20:
                discount += item.total() * .1
        return discount


def large_order_promo(order):
        """7% discount for orders including at least 10 different items"""
        distinct_items = {item.product for item in order.cart}
        if len(distinct_items) >= 10:
            return order.total() * .07
        return 0

In [6]:
joe = Customer('John Doe', 0)
ann = Customer('Ann Smith', 1100)
cart = [LineItem('banana', 4, .5),
        LineItem('apple', 10, 1.5),
        LineItem('watermelon', 5, 5.0)]
Order(joe, cart, fidelity_promo)

<Order total: 42.00 due: 42.00>

In [7]:
Order(ann, cart, fidelity_promo)

<Order total: 42.00 due: 39.90>

In [8]:
banana_cart = [LineItem('banana', 30, .5),
               LineItem('apple', 10, 1.5)]
Order(joe, banana_cart, bulk_item_promo)

<Order total: 30.00 due: 28.50>

In [10]:
long_order = [LineItem(str(item_code), 1, 1.0) for item_code in range(10)]
Order(joe, long_order, large_order_promo)

<Order total: 10.00 due: 9.30>

In [11]:
Order(joe, cart, large_order_promo)

<Order total: 42.00 due: 42.00>

#### Пример 6.6. 
best_promo находит максимальную скидку, перебирая все функции

In [14]:
promos = [fidelity_promo, bulk_item_promo, large_order_promo]

def best_promo(order):
    """Choose max possible discount"""
    return max(promo(order) for promo in promos)

#### Пример 6.5. 
Функция best_promo применяет все стратегии и возвращает наибольшую скидку

In [15]:
Order(joe, long_order, best_promo)

<Order total: 10.00 due: 9.30>

In [16]:
Order(joe, banana_cart, best_promo)

<Order total: 30.00 due: 28.50>

In [17]:
Order(ann, cart, best_promo)

<Order total: 42.00 due: 39.90>

#### Пример 6.7. 
Список promos строится путем просмотра глобального пространства
имен модуля

In [18]:
promos = [globals()[name] for name in globals()
          if name.endswith('_promo') and name != 'best_promo']
def best_promo(order):
    """Choose max possible discount"""
    return max(promo(order) for promo in promos)

#### Пример 6.8. 
Список promos строится путем интроспекции нового модуля promotions

Функция inspect.getmembers возвращает атрибуты объекта – в данном случае
модуля promotions – возможно, отфильтрованные предикатом (булевой функцией). Мы пользуемся предикатом inspect.isfunction, чтобы получить только имеющиеся в модуле функции

In [23]:
promotions = "This is module in this thera are all promotion functions"

In [24]:
import inspect
promos = [func for name, func in inspect.getmembers(promotions, inspect.isfunction)]
def best_promo(order):
    """Choose max possible discount"""
    return max(promo(order) for promo in promos)

### Паттерн Команда